# Analysis of Pandemic - Part I

![board](images\diameter_path.png)

Next graphic shows a good research station placed, the new edge from atlanta to new RS, and the new shortest path between those cities. 

![board](images\bad_rs_placement.png)

Next graphic shows a good research station placed, the new edge from atlanta to new RS, and the new shortest path between those cities. 

![board](images\good_rs_placement.png)

See _Pandemic Analysis - Generate data.ipynb_    

<div style="margin:500px"></div>

In [ ]:
import json
import pandas as pd
import networkx as nx
#import pandemic
import itertools
from collections import Counter

In [ ]:
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 150)

## Load Pandemic board as graph

In [ ]:
pandemic_g = nx.read_graphml('pandemic.graphml.txt')

# dicts from city names to numbers, and from numbers to city names.
city_names_to_num = {tup[1]['label']:tup[0] for tup in pandemic_g.nodes(data=True)}
city_num_to_names = {tup[0]:tup[1]['label'] for tup in pandemic_g.nodes(data=True)}

###### Calculate starting diameter

In [ ]:
nx.diameter(pandemic_g)

## Load data 

In [ ]:
# Loop through all the json results files, load them into dataframe, rename the columns, 
# and concat them all together.
df = pd.concat([pd.read_json('{}_nodes.json'.format(i)).
                rename(columns={0:'Diameter', 1:'Stations', 2:'Num_Edges'})
                for i in xrange(1,6)],
                ignore_index=True).drop('Num_Edges', axis=1)
# Add column for number of stations per combination.
df['Num_Stations'] = df.Stations.map(lambda x: len(x))
# Add column with names of staitons.
df['City_Names'] = df.Stations.map(lambda stations: [city_num_to_names[s] for s in stations])

In [ ]:
df.shape

In [ ]:
df.sample(10)

## Absolute minimum diameter

In [ ]:
df.Diameter.min()

In [ ]:
abs_min_dia = df[df.Diameter==5]
abs_min_dia

###### Number of station combos with min diameter

In [ ]:
len(abs_min_dia)

###### Minimum number of stations needed to get min diameter

In [ ]:
abs_min_dia.Num_Stations.min()

So all of the minimum combos have six stations.  
You cannot get a diameter of five with fewer stations.

<div style="margin:500px"></div>

## Minimum diameter by number of stations in combo

###### What is the minimum diameter for each number of stations?

In [ ]:
# Get the minimum diameter by number of stations.
pd.DataFrame(df.groupby('Num_Stations')['Diameter'].min())  # Convert to dataframe for easier reading.

###### How many combos yield the minimum diameter by number of stations?

In [ ]:
# Get the number of combos with the minimum diameter by number of stations.
(pd.concat([df.groupby('Num_Stations')['Diameter'].min(),
           df.groupby('Num_Stations').apply(lambda grp: len(grp[grp.Diameter==grp.Diameter.min()])), 
           df.groupby('Num_Stations').size()],
         axis=1).rename(columns={'Diameter':'Min diameter', 0:'Combos with min diameter', 1:'Total combos'})
 [['Total combos', 'Combos with min diameter','Min diameter']])

###### Filter for just the combinations that give the minimum diamater by number of stations

In [ ]:
min_dia_by_num_of_stations = df.groupby('Num_Stations').apply(lambda grp: grp[grp.Diameter==grp.Diameter.min()]).drop('Stations', axis=1)
min_dia_by_num_of_stations.drop('Num_Stations', axis=1)

###### Most common cities in minimum combos by number of stations

In [ ]:
most_common_in_min_combos_by_num_of_stats = (min_dia_by_num_of_stations.groupby('Num_Stations')
                        .apply(lambda grp: Counter(itertools.chain(*grp.City_Names))))

In [ ]:
most_common_in_min_combos_by_num_of_stats[2]

In [ ]:
most_common_in_min_combos_by_num_of_stats[3].most_common()

We can see groups start to form of cities clustered together that represent the same area of the graph and probably appear in separate optimal combos:  
Istanbul, Baghdad, Cairo  
Hong Kong, Shanghai, Taipei

![board](images/3_city_groups.png)

In [ ]:
min_dia_by_num_of_stations[min_dia_by_num_of_stations.City_Names.map(lambda grp: u'S\xe3o Paulo' in grp)].loc[3]

![board](images/sao_paulo_3_combos.png)

In [ ]:
most_common_in_min_combos_by_num_of_stats[4].most_common(15)

In [ ]:
most_common_in_min_combos_by_num_of_stats[5].most_common(15)

In [ ]:
most_common_in_min_combos_by_num_of_stats[6].most_common(15)

###### Which cities appear in the most combos across all groups?

In [ ]:
# First, calculate the fractions within each number of stations.
cities_frac = []
for num_of_stations in xrange(2,7):
    c = most_common_in_min_combos_by_num_of_stats[num_of_stations]
    #cities = map(lambda tup: (pandemic.city_num_to_names[tup[0]], tup[1]), c.most_common(None))
    cities = c.most_common(None)
    total_combos = float(cities[0][1])
    for city in cities:
        cities_frac.append((city[0], city[1], city[1]/total_combos, num_of_stations))

frac_df = pd.DataFrame.from_records(cities_frac, columns=['City', 'Occurrences', 'Fraction', 'Num_Stations'])
frac_df = frac_df.set_index(['Num_Stations', 'City'])
frac_df

In [ ]:
# Sum fractions for each city to get a metric we can use to rank the cities.
rankings = (pd.DataFrame(frac_df.reset_index()
                         .groupby('City')['Fraction']
                         .sum()
                         .sort_values(ascending=False))
            .reset_index()).rename(columns={'Fraction':'Weighted Sum'})
# Add the degree of each city.
rankings['Degree'] = rankings.City.map(lambda c: pandemic_g.degree(city_names_to_num[c]))
rankings

We are summing fractions from separate groups, so the total is not a fraction but more like a weighted sum. The thought is that simply counting the number of optimal combos each city appears in could be biased toward cities that are part of combos with more stations (4, 5, 6 stations). Instead, we want to know the normalize frequency with which each city appears in an optimal combo for each group (number of stations).

In [ ]:
# For each city, how many optimal combos does it appear in for each number of stations.
city_optimals_by_num_of_stations = frac_df.drop('Fraction', axis=1).unstack('Num_Stations')
city_optimals_by_num_of_stations.columns = city_optimals_by_num_of_stations.columns.droplevel(0)
city_optimals_by_num_of_stations = city_optimals_by_num_of_stations.fillna(0).astype(int)
city_optimals_by_num_of_stations['Total'] = city_optimals_by_num_of_stations.sum(axis=1)
# Merge the rankings with the above dataframe.
rankings = rankings.merge(city_optimals_by_num_of_stations, how='left', left_on='City', right_index=True)
rankings

So Hong Kong and Cairo are about 2x as important as the next cities on the list, Baghdad and Mexico City.

![board](images\hk_cairo.png)

<div style="margin:500px"></div>

## Determine which optimal combos are subsets of other optimal combos

In [ ]:
optimal_subsets = min_dia_by_num_of_stations.reset_index(level=1,drop=True).assign(City_Names=lambda df: df.City_Names.map(frozenset))

In [ ]:
# Calculate combos that are subsets of larger combos.
sub_of = {}
for i in optimal_subsets.index.unique()[:-1]:
    for sub_combo in optimal_subsets.City_Names.loc[i]:
        sub_of[sub_combo] = [c for c in optimal_subsets.City_Names.loc[i+1] if sub_combo.issubset(c)]
sub_of = {k:v for k,v in sub_of.iteritems() if len(v) > 0}                

In [ ]:
# Filter down to combos of less than 4 for easier viewing.
{k:v for k,v in sub_of.items() if len(k) < 4}

###### How many combinations are subsets of larger combinations by number of stations

In [ ]:
c = Counter([len(s) for s in sub_of])
pd.DataFrame(c.items(), columns=['Number of Stations', 'Number of combos that are subsets of a larger combo'])

In [ ]:
sorted([sorted(list(s)) for s in sub_of if len(s) < 3], key=lambda x: len(x))

In [ ]:
sorted([sorted(list(s)) for s in sub_of if len(s) < 4], key=lambda x: len(x))

###### Build graph to find longest chain of subsetting optimal combos

In [ ]:
optimal_combos_graph = nx.DiGraph()
optimal_combos_graph.add_edges_from(itertools.chain.from_iterable([itertools.product([k], v) for k,v in sub_of.iteritems()]))

In [ ]:
nx.descendants(optimal_combos_graph, frozenset({u'Atlanta', u'Cairo'}))

In [ ]:
# Add 'sink' node to making calculating chains easier.
for n in optimal_combos_graph.nodes():
    if len(n) == 6:
        optimal_combos_graph.add_edge(n, 'sink')

In [ ]:
longest_combo_chains = list(nx.all_shortest_paths(optimal_combos_graph, frozenset({u'Atlanta', u'Cairo'}), 'sink'))

In [ ]:
len(longest_combo_chains)

In [ ]:
chain = longest_combo_chains[0]
print(list(chain[0]))
for i in xrange(1, len(chain)-1):
    print(list(chain[i] - chain[i-1])[0])

In [ ]:
longest_combo_chains[0]